# Create an empty Ubuntu VM on Azure

Install HAGrid

```
$ pip install hagrid
```

__NOTE__: Just in case there are some issues with a VM we should provision are few extra, here we use node_count 12 for a session of 10 users

Run hagrid launch with these arguments:

```
$ hagrid launch to azure --image_name=domain_0.7.0 --jupyter --ansible_extras="install=false" --node_count 12
```

What ever you enter into Repo and Branch will be ignored

![ip_address](img/hagrid_bare_vm.png)

After it is finished you should see this message

![ip_address](img/hagrid_bare_vm_output.png)

Now run this to get JSON containing the information for all the VMs:

```
$ cat ~/.hagrid/host_ips.json
```

In [7]:
import os
import json

# paste the path to host ip json here
HOST_IP_PATH = "~/.hagrid/host_ips.json"
HOST_IP_PATH = os.path.expanduser(HOST_IP_PATH)

with open(HOST_IP_PATH) as fp:
    host_ips = json.loads(fp.read())

In [8]:
host_ips

{'host_ips': [{'username': 'azureuser',
   'password': ')!%IW+*1yvueE9U8',
   'ip_address': '20.66.107.124',
   'jupyter_token': '5nhpot6k1uzzd02mwrawywamc6vmkynpylo0l86c8e4hv708'},
  {'username': 'azureuser',
   'password': ')!%IW+*1yvueE9U8',
   'ip_address': '20.245.75.55',
   'jupyter_token': 'jfk6c76vz0mt9svqtjezxpdqzssoaks47jblybjgh85wyre2'},
  {'username': 'azureuser',
   'password': ')!%IW+*1yvueE9U8',
   'ip_address': '20.245.75.78',
   'jupyter_token': '5twgz5m1yf3fsav7rq2xdrcknjyr2l3tebqoxexip5nolyyl'},
  {'username': 'azureuser',
   'password': ')!%IW+*1yvueE9U8',
   'ip_address': '20.66.104.212',
   'jupyter_token': 'b4w4ccmxrx18i5m1g7wfsuc480s1p94cni3djd952fedrt3c'},
  {'username': 'azureuser',
   'password': ')!%IW+*1yvueE9U8',
   'ip_address': '20.245.75.77',
   'jupyter_token': 'f7e245ukd2ieh9zeuqmji65ddyid32z5fdnvw8c85x9c7k3x'},
  {'username': 'azureuser',
   'password': ')!%IW+*1yvueE9U8',
   'ip_address': '20.245.75.79',
   'jupyter_token': '7myi5231sv9zcf2trhpf8okz

In [9]:
# update TOTAL_PARTICIPANTS
# use the total participants not the total machines, e.g. 10 not 12
# as this is used to calculate the data split assignment
TOTAL_PARTICIPANTS = 10

In [12]:
# optionally add names or emails here which will be printed below to help keep track of assignment
participants = [
    "Teo",
    "Ruchi",
    "Kyoko",
    "Ivy",
    "Shubham",
    "Irina",
    "Laura",
    "Ionesio",
    "Ronnie",
    "Rasswanth"
]
print("Total participants:", len(participants))

Total participants: 10


In [13]:
def check_ip_port(host_ip: str, port: int) -> bool:
    import socket
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((host_ip, port))
        sock.close()
        if result == 0:
            return True
    except Exception:
        pass
    return False

In [14]:
def get_icon(status: bool) -> str:
    return "✅" if status else "❌"

In [15]:
def check_hosts_ready(host_ips: dict) -> None:
    for host in host_ips["host_ips"]:
        print("-----------------------")
        host_ip = host["ip_address"]
        # make sure the containers are not running
        http_up = check_ip_port(host_ip=host_ip, port=80)
        print(f"{get_icon(not http_up)} Containers Off {host_ip}:80")
        
        # make sure jupyter notebooks is up
        jupyter_up = check_ip_port(host_ip=host_ip, port=8888)
        print(f"{get_icon(jupyter_up)} Jupyter Up {host_ip}:8888")
        
        # make sure SSH is up
        ssh_up = check_ip_port(host_ip=host_ip, port=22)
        print(f"{get_icon(ssh_up)} SSH Up {host_ip}:22")

        print()
        all_status = (not http_up) and jupyter_up and ssh_up
        print(f"{get_icon(all_status)} Node {host_ip} Ready!")
        print("-----------------------")
        print()

In [19]:
check_hosts_ready(host_ips)

-----------------------
❌ Containers Off 20.66.107.124:80
✅ Jupyter Up 20.66.107.124:8888
✅ SSH Up 20.66.107.124:22

❌ Node 20.66.107.124 Ready!
-----------------------

-----------------------
❌ Containers Off 20.245.75.55:80
✅ Jupyter Up 20.245.75.55:8888
✅ SSH Up 20.245.75.55:22

❌ Node 20.245.75.55 Ready!
-----------------------

-----------------------
❌ Containers Off 20.245.75.78:80
✅ Jupyter Up 20.245.75.78:8888
✅ SSH Up 20.245.75.78:22

❌ Node 20.245.75.78 Ready!
-----------------------

-----------------------
❌ Containers Off 20.66.104.212:80
✅ Jupyter Up 20.66.104.212:8888
✅ SSH Up 20.66.104.212:22

❌ Node 20.66.104.212 Ready!
-----------------------

-----------------------
❌ Containers Off 20.245.75.77:80
✅ Jupyter Up 20.245.75.77:8888
✅ SSH Up 20.245.75.77:22

❌ Node 20.245.75.77 Ready!
-----------------------

-----------------------
❌ Containers Off 20.245.75.79:80
✅ Jupyter Up 20.245.75.79:8888
✅ SSH Up 20.245.75.79:22

❌ Node 20.245.75.79 Ready!
---------------------

In [8]:
def output_user_details(host_ips: dict, participants: list[str] = []) -> None:
    notebook_path = "adastra/data-owners/00-data-owners-create-domain.ipynb"
    print("===============================")
    print("Ad Astra 🚀🌌")
    print("===============================")
    print()
    print("Send to each participant")
    print()
    if TOTAL_PARTICIPANTS > len(host_ips["host_ips"]):
        raise Exception(f"TOTAL_PARTICIPANTS: {TOTAL_PARTICIPANTS} is less than VM count: {len(host_ips['host_ips'])}")
    partition = 0
    for host in host_ips["host_ips"]:
        partition += 1
        if partition <= len(participants):
            print(f"Hi {participants[partition - 1]},")
        if partition <= TOTAL_PARTICIPANTS:
            print("These are your Session Details:")
        else:
            print("Spare Session Details:")
        print("-------------------------------")
        print(f"Username: {host['username']}")
        print(f"Password: {host['password']}")
        print(f"VM IP Address: {host['ip_address']}")
        if partition <= TOTAL_PARTICIPANTS:
            print()
            print(f"MY_PARTICIPANT_NUMBER={partition}")
            print(f"TOTAL_PARTICIPANTS={TOTAL_PARTICIPANTS}")

        print()
        print(f"👉🏽 Start Here:")
        print(
            f"http://{host['ip_address']}:8888/lab/tree/notebooks/{notebook_path}"
            f"?token={host['jupyter_token']}"
        )
        print()

In [9]:
output_user_details(host_ips, participants)

Ad Astra 🚀🌌

Send to each participant

Hi Andrew,
These are your Session Details:
-------------------------------
Username: azureuser
Password: ewwAVCwn*I_XJhy8
VM IP Address: 20.228.100.98

MY_PARTICIPANT_NUMBER=1
TOTAL_PARTICIPANTS=2

👉🏽 Start Here:
http://20.228.100.98:8888/lab/tree/notebooks/adastra/data-owners/00-data-owners-create-domain.ipynb?token=rwxl7mkbgn4an2soi0zktesshljcv03evgfkhjtkmrxwaifn

These are your Session Details:
-------------------------------
Username: azureuser
Password: ewwAVCwn*I_XJhy8
VM IP Address: 20.228.100.99

MY_PARTICIPANT_NUMBER=2
TOTAL_PARTICIPANTS=2

👉🏽 Start Here:
http://20.228.100.99:8888/lab/tree/notebooks/adastra/data-owners/00-data-owners-create-domain.ipynb?token=y83at4izkwus70mpi0sbawi92tk5xwq2ddwodh7rp42eyu5m

